### 1. Settings

In [1]:
#####################################
##########  DEPENDECIES ############
#####################################

import os
import pickle
import sys
sys.path.append("../")

from tqdm import tqdm # type: ignore
from datetime import date

import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from utils.sampling import create_splits, prep_cv_validation
from utils.training import cv_cluster_set, cv_training_epochs_sets, test_cluster_set
from utils.training import results_dict_todf, cv_step_2, full_step_2, test_training_epochs_sets
from utils.inference import meta_predict, create_ensemble_map, ensemble_compute

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch

#####################################
############  CONSTANTS #############
#####################################

RS = 42
BATCH_SIZE = 16
DECODER_LENGTH = 30
ENCODER_LENGTH = 30
MODEL_NAME = "Salesforce/codet5-base-multi-sum"

FULL_TRAIN_ARGS = {
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "SEQ_TRAINER_ARGS": {
        "overwrite_output_dir": True,
        "num_train_epochs": [0, 1, 2, 5, 10],
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 1e-5,
        "warmup_steps": 500,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "no",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
        "output_dir" : 'reports/results',
        "logging_dir" : "reports/logs",
    },
}

experiment_config = {
    "DATA_STR" : "20240908",
    "RS" : RS,
    "DRIFT_TYPE" : "no_drift",
    "NFOLD" : 3,
    "FULL_TRAIN_ARGS" : FULL_TRAIN_ARGS,
    "MODEL_NAME" : MODEL_NAME,
    "CLUSTER_EPOCHS" : 3,
    "CLUSTER_SET_ID" : [0, 3, [0, 3,]],
    "TRAIN_SIZE" : 7000,
    "TEST_SIZE" : 2500,
}
experiment_config["ANALYSIS_POSTFIX"] = f"mined_{experiment_config['DRIFT_TYPE']}_{str(date.today())}"
experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"] += "/" + experiment_config["ANALYSIS_POSTFIX"] 
experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"] += "/" + experiment_config["ANALYSIS_POSTFIX"] 

if not os.path.exists(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"]):
    os.mkdir(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"])

if not os.path.exists(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"]):
    os.mkdir(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"])

tokenizer = AutoTokenizer.from_pretrained(experiment_config["MODEL_NAME"], skip_special_tokens=False)
model = AutoModelForSeq2SeqLM.from_pretrained(experiment_config["MODEL_NAME"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rouge = evaluate.load('rouge')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Conala data. Preprocessing. 

In [2]:
sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, train_size=experiment_config["TRAIN_SIZE"], test_size=experiment_config["TEST_SIZE"], cluster_id=4)
train_dataset, test_data, test_df, train_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"], sampling_dict["train_df"]

splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
                            experiment_config=experiment_config)

Train Data:  (7000, 11)
Test Data:  (2500, 11)
Train Data: Cluster cluster
1    3280
0    1352
2    1325
3    1043
Name: count, dtype: int64
Test Data: Cluster cluster
1    1156
2     490
0     441
3     413
Name: count, dtype: int64


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Fold 0
Fold 1
Fold 2


In [3]:
fold_results = cv_training_epochs_sets(experiment_config=experiment_config,
                            splits=splits,
                            questions_list=questions_list,
                            train_dataset=train_dataset,
                            tokenizer=tokenizer)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_fold_epoch_set.pickle', 'wb') as handle:
    pickle.dump(fold_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

Fold 0
/usr/net/zinovyee.hub/IRTG/MLSC/MLSC_DD/ensemble_learning


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Fold train:  (4666, 14)
Fold val:  (2334, 14)
TRAINING EPOCH SET 0
TRAINING EPOCHS 0
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.245800,4.464506,0.278500,0.077600,0.245800,0.246200,13.424800,0.028100,1.000000,1.082700,24282,22427
2,4.388000,4.427325,0.280000,0.079000,0.245700,0.246200,13.776700,0.027600,1.000000,1.114800,25001,22427
3,4.281000,4.424661,0.282300,0.080400,0.248600,0.249100,13.633500,0.029400,1.000000,1.096700,24595,22427


Generating Summaries
Computing Performance
Saving the model
TRAINING EPOCH SET 10
TRAINING EPOCHS 5
LOADING MODEL ./models/tmp/
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.777100,4.466114,0.275800,0.077300,0.242300,0.242500,13.185600,0.029200,1.000000,1.051200,23575,22427
2,3.988500,4.446638,0.279100,0.077600,0.244500,0.244900,13.643800,0.028500,1.000000,1.095800,24576,22427
3,4.009800,4.422835,0.279900,0.079000,0.246400,0.246600,13.648500,0.028700,1.000000,1.092500,24501,22427
4,4.002400,4.429433,0.282100,0.079900,0.247300,0.247700,13.509600,0.028700,1.000000,1.078100,24179,22427
5,3.960300,4.433535,0.281200,0.079600,0.246300,0.246700,13.661400,0.027800,1.000000,1.094300,24541,22427


Generating Summaries
Computing Performance
Saving the model


In [4]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_fold_epoch_set.pickle', 'rb') as handle:
   fold_results = pickle.load(handle)
   
for cluster_idx in experiment_config["CLUSTER_SET_ID"]:
    fold_results = cv_cluster_set(experiment_config=experiment_config,
                                            splits=splits,
                                            questions_list=questions_list,
                                            train_dataset=train_dataset,
                                            tokenizer=tokenizer,
                                            fold_results=fold_results,
                                            cluster_id=cluster_idx)

cv_df = results_dict_todf(fold_results)

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_step1.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Fold 0


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Cluster [0] training size (890, 14)
TRAINING CLUSTER SET [0] FOR EPOCHS 3


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Cluster [0, 3] training size (1592, 14)
TRAINING CLUSTER SET [0, 3] FOR EPOCHS 3


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


LOADING MODEL ./models/tmp/
cuda


In [5]:
print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())

Mean
model_set
0                 0.130373
1                 0.269827
2                 0.279207
5                 0.284077
10                0.284135
cluster_[0, 3]    0.270036
cluster_[0]       0.198179
cluster_[3]       0.249134
Name: rouge, dtype: float64
STD
model_set
0                 0.137666
1                 0.160884
2                 0.162966
5                 0.164087
10                0.164493
cluster_[0, 3]    0.160769
cluster_[0]       0.161238
cluster_[3]       0.150450
Name: rouge, dtype: float64


### Step 2. Learn performance

In [6]:
########## LOAD CV RESULTS

import pickle
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_step1.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

########## RUN STEP 2 ON CV

cv_df, model_results = cv_step_2(experiment_config=experiment_config, cv_df=cv_df, t_models=["lgbm"])

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/s2_model_results.pickle', 'wb') as handle:
    pickle.dump(model_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_results.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)


0
lgbm
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024430 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13018
[LightGBM] [Info] Number of data points in the train set: 37328, number of used features: 1024
[LightGBM] [Info] Start training from score 0.245488
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

In [7]:
### TO SAVE THE VECTORIZER AND STEP 2 MODELS

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_results.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)


print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())


full_step_2(cv_df=cv_df, 
            experiment_config=experiment_config, t_models=["lgbm"])

Mean
model_set
0                 0.130373
1                 0.269827
2                 0.279207
5                 0.284077
10                0.284135
cluster_[0, 3]    0.270036
cluster_[0]       0.198179
cluster_[3]       0.249134
ensemble          0.284177
Name: rouge, dtype: float64
STD
model_set
0                 0.137666
1                 0.160884
2                 0.162966
5                 0.164087
10                0.164493
cluster_[0, 3]    0.160769
cluster_[0]       0.161238
cluster_[3]       0.150450
ensemble          0.164489
Name: rouge, dtype: float64
lgbm
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[L

# TEST

In [8]:
# sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, test=True, train_size=100, test_size=25, cluster_id=4)
# train_dataset, test_data, test_df, train_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"], sampling_dict["train_df"]

# splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
#                             experiment_config=experiment_config)

In [9]:
with open(f"reports/results/{experiment_config['ANALYSIS_POSTFIX']}/cv_results.pickle", "rb") as handle:
    cv_resutls = pickle.load(handle)

base_models_list = list(cv_resutls.model_set.unique())
base_models_list.pop(-1)

'ensemble'

In [10]:
meta_preds_df = meta_predict(experiment_config=experiment_config, 
                    test_df=test_df,
                    base_models_names=base_models_list,
                    t_models=["lgbm"])

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_step2.pickle', 'wb') as handle:
    pickle.dump(meta_preds_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

lgbm
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [11]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_step2.pickle', 'rb') as handle:
    meta_preds_df = pickle.load(handle)
    
meta_preds_df.groupby("model_set").lgbm_preds.mean()

model_set
0                 0.138374
1                 0.272835
2                 0.277185
5                 0.279695
10                0.279916
cluster_[0, 3]    0.272846
cluster_[0]       0.206853
cluster_[3]       0.256394
Name: lgbm_preds, dtype: float64

In [12]:
optimal_ensemble_map, ensemble_val_estim = create_ensemble_map(meta_preds_df=meta_preds_df, 
                                                                t_model_name="lgbm")

In [13]:
test_result_df = test_training_epochs_sets(experiment_config=experiment_config,
                            test_df=test_df,
                            test_data=test_data,
                            train_data=train_dataset,
                            tokenizer=tokenizer)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_epoch_set.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

TRAINING EPOCH SET 0
TRAINING EPOCHS 0
Salesforce/codet5-base-multi-sum


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


cuda
TRAINING EPOCH SET 1
TRAINING EPOCHS 1
Salesforce/codet5-base-multi-sum


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


cuda


...

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.207900,4.426007,0.282800,0.075600,0.245900,0.245900,13.785600,0.026900,1.000000,1.102000,26916,24424
2,4.339300,4.395939,0.282100,0.077300,0.247900,0.248200,13.762800,0.029700,1.000000,1.086500,26536,24424
3,4.245400,4.392785,0.284600,0.078000,0.249000,0.249200,13.714800,0.030500,1.000000,1.081400,26413,24424


TRAINING EPOCH SET 10
TRAINING EPOCHS 5
./models/5_epoch_set


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


cuda


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.746500,4.423030,0.282100,0.073200,0.245700,0.245900,13.685200,0.025900,1.000000,1.086900,26546,24424
2,3.968400,4.407227,0.283800,0.078700,0.249300,0.249400,13.738800,0.029000,1.000000,1.077800,26324,24424
3,3.991700,4.390144,0.282700,0.075200,0.247000,0.247300,13.742400,0.027600,1.000000,1.077700,26321,24424
4,3.979800,4.394220,0.279000,0.074200,0.243100,0.243200,13.652000,0.026500,1.000000,1.070000,26134,24424
5,3.931500,4.400657,0.280700,0.075600,0.246000,0.246100,13.696000,0.028000,1.000000,1.069500,26122,24424


In [14]:
train_df.cluster.value_counts()

cluster
1    3280
0    1352
2    1325
3    1043
Name: count, dtype: int64

In [15]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_epoch_set.pickle', 'rb') as handle:
   test_result_df = pickle.load(handle)

test_result_df = test_result_df.rename(columns={"epoch_set": "model_set"})

for cluster_idx in experiment_config["CLUSTER_SET_ID"]:
    test_result_df = test_cluster_set(experiment_config=experiment_config,
                                    test_df=test_df,
                                    test_data=test_data,
                                    tokenizer=tokenizer,
                                    results_df=test_result_df,
                                    cluster_id=cluster_idx,
                                    train_df=train_df)

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Cluster [0] training size (1352, 14)
TRAINING CLUSTER SET [0] FOR EPOCHS3


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.755900,5.176994,0.150700,0.018700,0.138800,0.139300,7.120400,0.004600,0.372700,0.503300,12292,24424
2,4.956500,4.982159,0.192500,0.032000,0.171500,0.171900,7.639200,0.004800,0.440000,0.549200,13413,24424
3,4.660500,4.959134,0.200000,0.033900,0.176500,0.177000,7.756000,0.006300,0.451500,0.557000,13605,24424


Cluster [3] training size (1043, 14)
TRAINING CLUSTER SET [3] FOR EPOCHS3
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


...

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.709300,5.176757,0.178400,0.033800,0.161700,0.161700,12.989200,0.012700,1.000000,1.058100,25843,24424
2,4.990300,4.840027,0.243800,0.061000,0.218300,0.218300,16.166800,0.021800,1.000000,1.344700,32842,24424
3,4.682900,4.773511,0.251200,0.065200,0.223900,0.224300,16.408400,0.022700,1.000000,1.366700,33381,24424


Cluster [0, 3] training size (2395, 14)
TRAINING CLUSTER SET [0, 3] FOR EPOCHS3


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.437800,4.739243,0.229700,0.056000,0.207300,0.207100,13.457200,0.018700,1.000000,1.085600,26515,24424
2,4.779100,4.603551,0.261300,0.066500,0.229900,0.230000,14.846400,0.021400,1.000000,1.217200,29729,24424
3,4.617500,4.581701,0.265700,0.068900,0.232600,0.232700,14.552400,0.022500,1.000000,1.193900,29160,24424


In [16]:
### ENSEMBLE COMPUTE
test_result_df = ensemble_compute(test_result_df=test_result_df,
                                  optimal_ensemble_map=optimal_ensemble_map)

########## ROUGE PER SETTING

print("Mean")
print(test_result_df.groupby("model_set")["rouge"].mean())

print("STD")
print(test_result_df.groupby("model_set")["rouge"].std())

print(test_result_df.opt_es_id.value_counts())

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results_full.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Mean
model_set
0                 0.123299
1                 0.276431
2                 0.282285
5                 0.286905
10                0.284072
cluster_[0, 3]    0.266360
cluster_[0]       0.202202
cluster_[3]       0.251622
ensemble          0.286127
Name: rouge, dtype: float64
STD
model_set
0                 0.133633
1                 0.162703
2                 0.162382
5                 0.165110
10                0.164032
cluster_[0, 3]    0.157822
cluster_[0]       0.158643
cluster_[3]       0.150236
ensemble          0.165175
Name: rouge, dtype: float64
opt_es_id
10    17541
5      3717
2       639
1       603
Name: count, dtype: int64


In [17]:
test_result_df.opt_es_id.value_counts()

opt_es_id
10    17541
5      3717
2       639
1       603
Name: count, dtype: int64